In [1]:
import os
import pandas as pd

import re

In [2]:

def extract_camera_timestamps(root_folder):
    # print("root folder: ",root_folder)
    for scene_folder in os.listdir(root_folder):
        # print("scene folder: ",scene_folder)
        scene_path = os.path.join(root_folder, scene_folder)
        # print("scene path: ",scene_path)
        if os.path.isdir(scene_path):
            camera_df = pd.DataFrame(columns=['camera_timestamp'])

            camera_folder_name = None
            for dir_name in os.listdir(scene_path):
                print("directory in scene_path",dir_name)
                if 'subset' in dir_name:
                    camera_folder_name = os.path.join(scene_path, dir_name)
                    # print("sss: ",camera_folder_name)
                    # print("www: ", camera_folder_name)
                    break
            # print("camera folder name: ",camera_folder_name)
            if camera_folder_name:
                for file in os.listdir(camera_folder_name):
                    if file.lower().endswith(('.jpg', '.jpeg')):
                        # Extract the camera timestamp from the file name using regular expressions
                        timestamp_match = re.search(r'\d{2}-\d{2}-\d{2}\.\d{6}', file)
                        
                        # timestamp_match = re.search(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}\.\d{6}', file)
                        if timestamp_match:
                            camera_timestamp = timestamp_match.group(0)
                            # print("rrrr: ",camera_timestamp)
                            # Remove the date part from the timestamp and replace '-' with ':'
                            camera_timestamp = camera_timestamp.replace('-', ':')
                            camera_df = camera_df.append({'camera_timestamp': camera_timestamp}, ignore_index=True)

            # Save camera timestamps CSV in the scene folder
            camera_timestamps_file = os.path.join(scene_path, 'camera_timestamps.csv')
            # print("camera_timestamps_file: ",camera_timestamps_file)
            camera_df.to_csv(camera_timestamps_file, index=False)

In [3]:
def extract_radar_timestamps(root_folder):
    # print("root folder: ",root_folder)
    for scene_folder in os.listdir(root_folder):
        # print("scene folder: ",scene_folder)
        scene_path = os.path.join(root_folder, scene_folder)
        # print("scene path: ",scene_path)
        if os.path.isdir(scene_path):
            radar_df = pd.DataFrame(columns=['radar_timestamp'])

            radar_folder_name = None
            for dir_name in os.listdir(scene_path):
                # print("directory in scene_path",dir_name)
                if '20' in dir_name:
                    radar_folder_name = os.path.join(scene_path, dir_name)
                    # print("ggggg: ",radar_folder_name)
                    break

            if radar_folder_name:
                for file in os.listdir(radar_folder_name):
                    # print("file: ",file)
                    if file.lower().endswith('.csv'):
                        # Extract the radar timestamp from the file name and replace '-' with ':'
                        radar_timestamp = os.path.splitext(file)[0].replace('-', ':')
                        radar_df = radar_df.append({'radar_timestamp': radar_timestamp}, ignore_index=True)

            # Save radar timestamps CSV in the scene folder
            radar_timestamps_file = os.path.join(scene_path, 'radar_timestamps.csv')
            radar_df.to_csv(radar_timestamps_file, index=False)

In [4]:
def synchronize_timestamps(camera_root_folder, radar_root_folder):
    for scene_folder in os.listdir(radar_root_folder):
        scene_path = os.path.join(radar_root_folder, scene_folder)
        # print(scene_path)
        if os.path.isdir(scene_path):
            # camera_timestamps_file = os.path.join(camera_root_folder, scene_folder, 'camera_timestamps.csv')
            camera_timestamps_file = os.path.join(camera_root_folder + '/cam_front','camera_timestamps.csv')

            # print('camera_root_folder',camera_root_folder)
            radar_timestamps_file = os.path.join(scene_path, 'radar_timestamps.csv')
            # print("aaaaaaa")
            # print("camera timestanp file",camera_timestamps_file)
            # print(radar_timestamps_file)
            if os.path.exists(camera_timestamps_file) and os.path.exists(radar_timestamps_file):
                # Read camera timestamps from the scene folder
                # print("TRUE")
                camera_df = pd.read_csv(camera_timestamps_file)
                radar_df = pd.read_csv(radar_timestamps_file)
                # print("camera_df: ",camera_df)
                # print("radar_df: ",radar_df)
                camera_df['camera_timestamp'] = pd.to_datetime(camera_df['camera_timestamp'], format='%H:%M:%S.%f')
                radar_df['radar_timestamp'] = pd.to_datetime(radar_df['radar_timestamp'], format='%H:%M:%S.%f')

                synchronized_df = pd.DataFrame(columns=['camera_timestamp', 'closest_radar_timestamp'])
                # print(synchronized_df)
                for camera_ts in camera_df['camera_timestamp']:
                    closest_radar_idx = (radar_df['radar_timestamp'] - camera_ts).abs().idxmin()
                    closest_radar_ts = radar_df.loc[closest_radar_idx, 'radar_timestamp']
                    synchronized_df = synchronized_df.append({'camera_timestamp': camera_ts, 'closest_radar_timestamp': closest_radar_ts}, ignore_index=True)

                synchronized_df['camera_timestamp'] = synchronized_df['camera_timestamp'].dt.strftime('%H:%M:%S.%f')
                synchronized_df['closest_radar_timestamp'] = pd.to_datetime(synchronized_df['closest_radar_timestamp']).dt.strftime('%H:%M:%S.%f')

                # Use the scene folder name in the output CSV file name
                synchronized_timestamps_file = os.path.join(scene_path, f'Radar_Camera_Synch_{scene_folder}.csv')
                print("synchronized_timestamps_file: ",synchronized_timestamps_file)
                synchronized_df.to_csv(synchronized_timestamps_file, index=False)

In [5]:
camera_root_folder = '/home/rad-cam/dataset_demo/camera'

# Specify the root folder for the radar
radar_root_folder = '/home/rad-cam/dataset_demo/radar'

# Call the functions to extract and synchronize timestamps
extract_camera_timestamps(camera_root_folder)
extract_radar_timestamps(radar_root_folder)
synchronize_timestamps(camera_root_folder, radar_root_folder)

directory in scene_path subset
synchronized_timestamps_file:  /home/rad-cam/dataset_demo/radar/front/Radar_Camera_Synch_123front.csv
